In [1]:
import lidar
import rosbag
from dask.distributed import Client
client = Client()  # start distributed scheduler locally.  Launch dashboard

Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


first converting the bagfile to a parquet file

In [6]:
bagfile = "/workspaces/lidar/tests/testdata/big.bag"
bagfile_bag = rosbag.Bag(bagfile)

In [2]:
#!/usr/bin/env python
import pandas as pd
import numpy as np
import itertools
from typing import Optional, List
import rosbag
from tqdm import tqdm
import sensor_msgs.point_cloud2 as pc2
from dask import delayed
import dask.dataframe as dd


PANDAS_TYPEMAPPING = {
    1: np.dtype("int8"),
    2: np.dtype("uint8"),
    3: np.dtype("int16"),
    4: np.dtype("uint16"),
    5: np.dtype("int32"),
    6: np.dtype("uint32"),
    7: np.dtype("float32"),
    8: np.dtype("float64"),
}



def read_bag(
    bag: rosbag.Bag,
    start_frame_number: Optional[int] = 0,
    end_frame_number: Optional[int] = None,
    keep_zeros: bool = False,
    topic: str = "/os1_cloud_node/points",
) -> List:
    messages = bag.read_messages(topics=[topic])
    sliced_messages = itertools.islice(messages, start_frame_number, None)
    result_list = []
    if end_frame_number is None:
        end_frame_number = 2  # TODO fix to lenght of messages
    for frame_number in tqdm(range(start_frame_number, end_frame_number, 1)):
        message = next(sliced_messages)
        frame = delayed(dataframe_from_message(message, keep_zeros))
        result_list.append(frame)
    return result_list


def dataframe_from_message(
    message: rosbag.bag.BagMessage, keep_zeros: bool = False
) -> pd.DataFrame:
    columnnames = [item.name for item in message.message.fields]
    type_dict = {
        item.name: PANDAS_TYPEMAPPING[item.datatype] for item in message.message.fields
    }
    frame_raw = pc2.read_points(message.message)
    frame_df = pd.DataFrame(np.array(list(frame_raw)), columns=columnnames)
    frame_df = frame_df.astype(type_dict)
    if not keep_zeros:
        frame_df = frame_df[
            (frame_df["x"] != 0.0) & (frame_df["y"] != 0.0) & (frame_df["z"] != 0.0)
        ]
        frame_df["original_id"] = frame_df.index
        frame_df = frame_df.astype({"original_id": "uint32"})
        frame_df = frame_df.reset_index(drop=True)
    return frame_df


  

In [30]:
lazy_dataframes = read_bag(bagfile_bag, 0, 200, False, "/os1_cloud_node/points")

100%|██████████| 200/200 [01:29<00:00,  2.22it/s]


In [9]:
test = dd.from_delayed(lazy_dataframes)

In [10]:
test.x.max().compute()

1.252312421798706

In [11]:
test.tail()

,x,y,z,intensity,t,reflectivity,ring,noise,range,original_id
111817,0.805429,0.045549,-0.115905,79.0,99926580,5,47,24,815,131055
111818,0.794220,-0.013478,-0.158536,108.0,99926580,7,53,39,810,131061
111819,0.803390,0.015773,-0.168396,101.0,99926580,7,54,58,821,131062
111820,0.762607,0.014853,-0.189751,69.0,99926580,4,58,37,786,131066
111821,0.776013,0.043899,-0.201706,64.0,99926580,4,59,32,803,131067


In [12]:
test.head()

,x,y,z,intensity,t,reflectivity,ring,noise,range,original_id
0,0.796484,-0.043220,0.236579,50.0,0,3,0,44,832,0
1,0.807739,0.046913,0.214302,134.0,0,9,3,42,837,3
2,0.808805,0.016405,0.190070,153.0,0,10,6,37,831,6
3,0.818806,-0.043872,0.177631,65.0,0,4,8,42,839,8
4,0.821585,-0.013810,0.169493,136.0,0,9,9,25,839,9


In [13]:
test

,x,y,z,intensity,t,reflectivity,ring,noise,range,original_id
npartitions=200,,,,,,,,,,
,float32,float32,float32,float32,uint32,uint16,uint8,uint16,uint32,uint32
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [14]:
lazy_dataframes[0].compute()

,x,y,z,intensity,t,reflectivity,ring,noise,range,original_id
0,0.796484,-0.043220,0.236579,50.0,0,3,0,44,832,0
1,0.807739,0.046913,0.214302,134.0,0,9,3,42,837,3
2,0.808805,0.016405,0.190070,153.0,0,10,6,37,831,6
3,0.818806,-0.043872,0.177631,65.0,0,4,8,42,839,8
4,0.821585,-0.013810,0.169493,136.0,0,9,9,25,839,9
...,...,...,...,...,...,...,...,...,...,...
112182,0.801167,0.016010,-0.137276,107.0,99939680,7,50,43,813,131058
112183,0.814400,0.046128,-0.147871,86.0,99939680,6,51,56,829,131059
112184,0.809908,-0.013744,-0.161668,107.0,99939680,7,53,39,826,131061
112185,0.795561,0.015620,-0.166755,86.0,99939680,6,54,57,813,131062


This is very promissing! I could just keep the tiny_lazy_dataframes and then convert it to a frame whenever needed!

What about the meta data, how can I use that?

In [17]:
test.to_parquet("/workspaces/export_test")

In [3]:
test2 = dd.read_parquet("/workspaces/export_test")

In [4]:
test2.x.max().compute()

1.252312421798706

Writing meta data as a json into the same folder

In [21]:
meta = {"oring_file": "sepp", "timestamps": [234234.234234, 234234.22234]}

In [22]:
import json

In [24]:
json.dump(meta)

TypeError: dump() missing 1 required positional argument: 'fp'

In [25]:
with open("/workspaces/export_test/data_file.json", "w") as write_file:
    json.dump(meta, write_file)


In [26]:
with open("/workspaces/export_test/data_file.json", "r") as read_file:
    data = json.load(read_file)


In [28]:
data["t"]

'sepp'

In [6]:
type(test2)

dask.dataframe.core.DataFrame

In [10]:
test2.describe().compute()

,x,y,z,intensity,t,reflectivity,ring,noise,range,original_id
count,2.240581e+07,2.240581e+07,2.240581e+07,2.240581e+07,2.240581e+07,2.240581e+07,2.240581e+07,2.240581e+07,2.240581e+07,2.240581e+07
mean,-1.395355e+00,-1.555028e-01,1.420729e-01,1.098774e+02,4.939794e+07,2.981485e+02,3.154140e+01,4.484404e+01,2.951956e+03,6.478594e+04
std,3.188425e+00,2.228653e+00,7.776557e-01,1.323687e+02,2.772368e+07,7.314155e+02,1.837981e+01,5.830621e+01,3.002505e+03,3.634500e+04
min,-3.452180e+01,-1.538729e+01,-6.654050e-01,8.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,6.650000e+02,0.000000e+00
25%,-1.821742e+00,-1.528600e+00,-2.560951e-01,4.700000e+01,2.574580e+07,7.000000e+00,1.600000e+01,3.000000e+01,1.212000e+03,3.375075e+04
50%,2.228281e-02,2.162177e-02,-4.399885e-03,7.100000e+01,5.035434e+07,2.100000e+01,3.200000e+01,3.900000e+01,1.956000e+03,6.606600e+04
75%,7.839448e-01,1.622289e+00,2.577102e-01,1.290000e+02,7.485591e+07,1.150000e+02,4.800000e+01,5.100000e+01,3.280000e+03,9.823450e+04
max,1.252312e+00,5.981112e+00,6.153335e+00,2.566000e+03,9.998461e+07,1.974000e+04,6.300000e+01,2.071000e+03,3.501700e+04,1.310700e+05


In [24]:
test2.shape

(Delayed('int-3bde0fd9-9481-440e-8069-0e92b3cb828e'), 10)

In [25]:
len(test2)

22405807

In [28]:
test2

,x,y,z,intensity,t,reflectivity,ring,noise,range,original_id
npartitions=200,,,,,,,,,,
,float32,float32,float32,float32,uint32,uint16,uint8,uint16,uint32,uint32
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [29]:
test2.npartitions

200

In [31]:
test3 = test2.get_partition(0)

In [32]:
type(test3)

dask.dataframe.core.DataFrame

In [37]:
test3.compute()

,x,y,z,intensity,t,reflectivity,ring,noise,range,original_id
0,0.796484,-0.043220,0.236579,50.0,0,3,0,44,832,0
1,0.807739,0.046913,0.214302,134.0,0,9,3,42,837,3
2,0.808805,0.016405,0.190070,153.0,0,10,6,37,831,6
3,0.818806,-0.043872,0.177631,65.0,0,4,8,42,839,8
4,0.821585,-0.013810,0.169493,136.0,0,9,9,25,839,9
...,...,...,...,...,...,...,...,...,...,...
112182,0.801167,0.016010,-0.137276,107.0,99939680,7,50,43,813,131058
112183,0.814400,0.046128,-0.147871,86.0,99939680,6,51,56,829,131059
112184,0.809908,-0.013744,-0.161668,107.0,99939680,7,53,39,826,131061
112185,0.795561,0.015620,-0.166755,86.0,99939680,6,54,57,813,131062


In [38]:
pd.Timestamp('2017-01-01T12')

Timestamp('2017-01-01 12:00:00')

In [39]:
type(pd.Timestamp('2017-01-01T12'))

pandas._libs.tslibs.timestamps.Timestamp

In [40]:
import datetime

In [45]:
type(datetime.datetime(2021,10,10))

datetime.datetime

In [48]:
datetime.datetime.now()

datetime.datetime(2021, 3, 12, 12, 53, 23, 276731)

In [56]:
test3 = [delayed(test2.get_partition(i)) for i in range(0,10)]

In [58]:
test4 = dd.from_delayed(test3)

In [59]:
test4.npartitions

10